In [1]:
import os
import numpy as np
os.chdir("/yourpath/CelebA")


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3,0" #list the gpu cores in the order you want to use e.g "0,1,2,3"

In [3]:
f=open('list_attr_celeba.txt').read()

In [4]:
Ft=f.split('\n')[1].split()

In [6]:
X=np.array([r.split() for r in f.split('\n')[2:-1]])
ids=X[:,0]

In [7]:
X.shape

(202599, 41)

## to load for CelebA-Age use the block below 

In [8]:
g=X[:,Ft.index('Young')+1]
y=np.array([X[:,i+1] for i in range(len(Ft)) if i!=Ft.index('Young')])
y=[[0 if t[i]=='-1' else 1 for i in range(len(t))] for t in y]
g=np.array([0 if g[i]=='-1' else 1 for i in range(len(g))])
no_tasks=[4,10,17,13,14,17,26,29,35]
y=np.array([y[t] for t in range(len(y)) if t not in no_tasks])
Ft.remove('Young')

## to load for CelebA-Gender use the block below instead of the block above

In [ ]:
g=X[:,Ft.index('Male')+1]
y=np.array([X[:,i+1] for i in range(len(Ft)) if i!=Ft.index('Male')])
y=[[0 if t[i]=='-1' else 1 for i in range(len(t))] for t in y]
g=np.array([0 if g[i]=='-1' else 1 for i in range(len(g))])
tasks=[2,3,5,6,7,8,11,12,19,20,22,24,26,30,31,32,38]
y=np.array([y[t] for t in tasks])
Ft.remove('Male')

## From here onwards all the blocks are common for both CelebA-Gender and CelebA-Age

In [15]:
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torchvision.models as pretrain
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [16]:
dataroot = "/yourpath/CelebA/celeba/"
image_size = 64
batch_size = 8192
workers = 2

dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

In [20]:
model_pre=pretrain.resnet18(pretrained=True)

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
cpu=torch.device('cpu')
dv=torch.device("cuda" if (torch.cuda.is_available() ) else "cpu")#
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
indices=np.arange(len(ids))
in_tr,in_val,in_ts=indices[:162770],indices[162770:182637],indices[182637:]
in_tr1=in_tr[:5*batch_size]; in_tr2=in_tr[5*batch_size:10*batch_size]; in_tr3=in_tr[10*batch_size:15*batch_size]
in_tr4=in_tr[15*batch_size:]
in_val1,in_val2=in_val[:10000],in_val[10000:]

dataset_tr1 = torch.utils.data.Subset(dataset, in_tr1)
dataset_tr2 = torch.utils.data.Subset(dataset, in_tr2)
dataset_tr3 = torch.utils.data.Subset(dataset, in_tr3)
dataset_tr4 = torch.utils.data.Subset(dataset, in_tr4)
dataset_val1 = torch.utils.data.Subset(dataset, in_val1)
dataset_val2 = torch.utils.data.Subset(dataset, in_val2)
dataset_ts = torch.utils.data.Subset(dataset, in_ts)

dataloader1 = torch.utils.data.DataLoader(dataset_tr1, batch_size=batch_size,
                                         shuffle=False, num_workers=workers)
dataloader2 = torch.utils.data.DataLoader(dataset_tr2, batch_size=batch_size,
                                         shuffle=False, num_workers=workers)
dataloader3 = torch.utils.data.DataLoader(dataset_tr3, batch_size=batch_size,
                                         shuffle=False, num_workers=workers)
dataloader4 = torch.utils.data.DataLoader(dataset_tr4, batch_size=batch_size,
                                         shuffle=False, num_workers=workers)

dataloader_val1 = torch.utils.data.DataLoader(dataset_val1, batch_size=10000,
                                         shuffle=False, num_workers=workers)
dataloader_val2 = torch.utils.data.DataLoader(dataset_val2, batch_size=10000,
                                         shuffle=False, num_workers=workers)

dataloader_test = torch.utils.data.DataLoader(dataset_ts, batch_size=len(in_ts),
                                         shuffle=False, num_workers=workers)


y_test=[y[i][in_ts] for i in range(len(y))]

y_v1,y_v2=[y[i][in_val1] for i in range(len(y))],[y[i][in_val2] for i in range(len(y))]

y_tr1,y_tr2=[y[i][in_tr1] for i in range(len(y))],[y[i][in_tr2] for i in range(len(y))]
y_tr3,y_tr4=[y[i][in_tr3] for i in range(len(y))],[y[i][in_tr4] for i in range(len(y))]



g_tr1,g_tr2,g_tr3,g_tr4=g[in_tr1],g[in_tr2],g[in_tr3],g[in_tr4]
g_val1,g_val2=g[in_val1],g[in_val2]
g_test=g[in_ts]
N_tasks=len(y)#4#2#3

In [33]:
d_v=[dataloader_val1,dataloader_val2]
yv=[y_v1,y_v2]
gv=[g_val1,g_val2]

In [28]:
for par in model_pre.parameters():
    par.requires_grad=False
in_fc=model_pre.fc.in_features
model_pre.fc=nn.Linear(in_fc,1024)

In [35]:
class STL(nn.Module):

    def __init__(self,pretrain):
        super(STL, self).__init__()
        self.fc1 = pretrain 
        self.bn1= nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 128)
        
        self.bn3 = nn.BatchNorm1d(1024)
        self.fc4 = nn.Linear(1024,1024)     
        self.task = nn.Linear(128,2)


    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        
        x = F.relu(self.bn3(self.fc4(x)))
        
        x = F.relu(self.bn2(self.fc2(x)))        
        x = F.relu(self.fc3(x))
        t = self.task(x)
       
        return t

In [36]:
def fair_loss(output,target,x_control):
    prot_att=x_control
    index_prot=torch.squeeze(torch.nonzero(prot_att[:] != 1.))
    target_prot=torch.index_select(target, 0, index=index_prot)
    index_prot_pos=torch.squeeze(torch.nonzero(target_prot[:] == 1. ))
    index_prot_neg=torch.squeeze(torch.nonzero(target_prot[:] == 0. ))

    index_non_prot=torch.squeeze(torch.nonzero(prot_att[:] == 1.))
    target_non_prot=torch.index_select(target, 0, index=index_non_prot)
    index_non_prot_pos=torch.squeeze(torch.nonzero(target_non_prot[:] == 1. ))
    index_non_prot_neg=torch.squeeze(torch.nonzero(target_non_prot[:] == 0. ))

    l_prot_pos=F.cross_entropy(torch.index_select(output, 0, index=index_prot_pos),torch.index_select(target, 0, index=index_prot_pos))    
    l_non_prot_pos=F.cross_entropy(torch.index_select(output, 0, index=index_non_prot_pos),torch.index_select(target, 0, index=index_non_prot_pos))    
    l_non_prot_neg=F.cross_entropy(torch.index_select(output, 0, index=index_non_prot_neg),torch.index_select(target, 0, index=index_non_prot_neg))
    l_prot_neg=F.cross_entropy(torch.index_select(output, 0, index=index_prot_neg),torch.index_select(target, 0, index=index_prot_neg))    

    for l in [l_prot_pos,l_non_prot_pos,l_prot_neg,l_non_prot_neg]:
        if torch.isinf(l)==True:
            l=torch.zeros_like(l,requires_grad=True)
    dl_pos=torch.max(l_prot_pos,l_non_prot_pos)
    dl_neg=torch.max(l_prot_neg,l_non_prot_neg)
    L=dl_pos+dl_neg
    
    return L

In [37]:
import torchmetrics
acc = torchmetrics.Accuracy()
def DM_rate(output,target,x_control):
    prot_att=x_control
    index_prot=torch.squeeze(torch.nonzero(prot_att[:] != 1.))
    target_prot=torch.index_select(target, 0, index=index_prot)
    index_prot_pos=torch.squeeze(torch.nonzero(target_prot[:] == 1. ))
    index_prot_neg=torch.squeeze(torch.nonzero(target_prot[:] == 0. ))

    index_non_prot=torch.squeeze(torch.nonzero(prot_att[:] == 1.))
    target_non_prot=torch.index_select(target, 0, index=index_non_prot)
    index_non_prot_pos=torch.squeeze(torch.nonzero(target_non_prot[:] == 1. ))
    index_non_prot_neg=torch.squeeze(torch.nonzero(target_non_prot[:] == 0. ))

    if index_prot_pos.shape==torch.Size([]) or index_prot_pos.shape==torch.Size([0])\
        or index_non_prot_pos.shape==torch.Size([]) or index_non_prot_pos.shape==torch.Size([0]):
            l_prot_pos=torch.tensor(0.0001)
            l_non_prot_pos=torch.tensor(0.0001)
    else:        
            l_prot_pos=acc(torch.index_select(output, 0, index=index_prot_pos),torch.index_select(target, 0, index=index_prot_pos))    
            l_non_prot_pos=acc(torch.index_select(output, 0, index=index_non_prot_pos),torch.index_select(target, 0, index=index_non_prot_pos))    
    
    if index_prot_neg.shape==torch.Size([]) or index_prot_neg.shape==torch.Size([0])\
        or index_non_prot_neg.shape==torch.Size([]) or index_non_prot_neg.shape==torch.Size([0]):
            l_prot_neg=torch.tensor(0.0001)
            l_non_prot_neg=torch.tensor(0.0001)
    else:        
            l_prot_neg=acc(torch.index_select(output, 0, index=index_prot_neg),torch.index_select(target, 0, index=index_prot_neg))    
            l_non_prot_neg=acc(torch.index_select(output, 0, index=index_non_prot_neg),torch.index_select(target, 0, index=index_non_prot_neg))  
            
    dl_pos=torch.abs(l_prot_pos-l_non_prot_pos)
    dl_neg=torch.abs(l_prot_neg-l_non_prot_neg)
    DM=dl_pos+dl_neg
    
    return DM

In [38]:
SLs=[nn.DataParallel(STL(model_pre.to(dv))).to(dv) for t in range(N_tasks)]
SL_optis=[optim.AdamW(SLs[t].parameters()) for t in range(N_tasks)]
spaths={'path'+str(t):'/yourpath/CelebA/model/Model_stl'+str(t)+'.pt' for t in range(N_tasks)}

In [40]:
criteria = nn.CrossEntropyLoss()
best_S=[[0,1] for t in range(N_tasks)]
dt=[dataloader1,dataloader2,dataloader3,dataloader4]
yt=[y_tr1,y_tr2,y_tr3,y_tr4]
gt=[g_tr1,g_tr2,g_tr3,g_tr4]
for epoch in range(20):  # loop over each NN multiple times
    loss_pointer=[]
    state_net,state_a,state_f=[[] for t in range(N_tasks)],[[] for t in range(N_tasks)],[[] for t in range(N_tasks)]
    R_A,R_F=[[] for t in range(N_tasks)],[[] for t in range(N_tasks)]
    ch=np.random.choice([0,1,2,3])
    dataloader,y_train,g_train=dt[ch],yt[ch],gt[ch]
    y_train=[torch.tensor(y_train[i]) for i in range(N_tasks)]
    xg=torch.tensor(g_train)
    for i, data in enumerate(dataloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs =data[0]
        if ((i+1)*batch_size)<len(in_tr):            
            labels=[y_train[t][i*batch_size:(i+1)*batch_size] for t in range(N_tasks)]#,y3_train[i:i+batch]]
            #if epoch<pretrn:
            xc=xg[i*batch_size:(i+1)*batch_size]
        else:
            labels=[y_train[t][i*batch_size:] for t in range(N_tasks)]#,y2_train[i:]]#,y3_train[i:]]
            #if epoch<pretrn:
            xc=xg[i*batch_size:]
        
        # zero the parameter gradients
        for t in range(N_tasks):
            ##training STLs
            SL_optis[t].zero_grad()
            out=SLs[t](inputs.to(dv).float())
            loss_Sa=criteria(out, labels[t].to(dv))
            loss_Sf=fair_loss(out, labels[t].to(dv),xc.to(dv))
            loss=loss_Sa+loss_Sf
            loss.backward()
            SL_optis[t].step()
            
            
            
            
    with torch.no_grad():
        ch=np.random.choice([0,1])
        val_batch,y_v,g_val=next(iter(d_v[ch]))[0],yv[ch],gv[ch]
       
        for t in range(N_tasks): 
            pred_SL=SLs[t](val_batch.to(dv).float())
            SL_acc=acc(pred_SL.to(cpu),torch.tensor(y_v[t]).to(cpu))
            SL_eo=DM_rate(pred_SL.to(cpu),torch.tensor(y_v[t]).to(cpu),torch.tensor(g_val).to(cpu))
            if SL_acc>best_S[t][0]:
                best_S[t][0]=SL_acc
                best_S[t][1]=SL_eo
                torch.save(SLs[t].state_dict(),spaths['path'+str(t)])
            print('Task',t,'  Acc:',SL_acc, 'EO:',SL_eo)         
        np.save(file="/yourpath/CelebA/Best_SL_new.npy",arr=np.array(best_S))

Task 0   Acc: tensor(0.6782) EO: tensor(0.0192)
Task 1   Acc: tensor(0.7888) EO: tensor(0.0193)
Task 2   Acc: tensor(0.8561) EO: tensor(0.0115)
Task 3   Acc: tensor(0.8454) EO: tensor(0.0211)
Task 4   Acc: tensor(0.7576) EO: tensor(0.0259)
Task 5   Acc: tensor(0.8248) EO: tensor(0.0313)
Task 6   Acc: tensor(0.7655) EO: tensor(0.0117)
Task 7   Acc: tensor(0.8591) EO: tensor(0.0134)
Task 8   Acc: tensor(0.6567) EO: tensor(0.0134)
Task 9   Acc: tensor(0.5973) EO: tensor(0.0112)
Task 10   Acc: tensor(0.9227) EO: tensor(0.0143)
Task 11   Acc: tensor(0.7197) EO: tensor(0.0019)
Task 12   Acc: tensor(0.7194) EO: tensor(0.0390)
Task 13   Acc: tensor(0.6577) EO: tensor(0.0249)
Task 14   Acc: tensor(0.7986) EO: tensor(0.0078)
Task 15   Acc: tensor(0.7664) EO: tensor(0.0190)
Task 16   Acc: tensor(0.7741) EO: tensor(0.0226)
Task 0   Acc: tensor(0.6847) EO: tensor(0.0287)
Task 1   Acc: tensor(0.7895) EO: tensor(0.0198)
Task 2   Acc: tensor(0.8636) EO: tensor(0.0086)
Task 3   Acc: tensor(0.8472) EO: 

Task 0   Acc: tensor(0.7042) EO: tensor(0.0128)
Task 1   Acc: tensor(0.7944) EO: tensor(0.0162)
Task 2   Acc: tensor(0.8745) EO: tensor(0.0126)
Task 3   Acc: tensor(0.8483) EO: tensor(0.0092)
Task 4   Acc: tensor(0.7692) EO: tensor(0.0213)
Task 5   Acc: tensor(0.8329) EO: tensor(0.0085)
Task 6   Acc: tensor(0.7850) EO: tensor(0.0092)
Task 7   Acc: tensor(0.8623) EO: tensor(0.0129)
Task 8   Acc: tensor(0.6811) EO: tensor(0.0056)
Task 9   Acc: tensor(0.6235) EO: tensor(0.0229)
Task 10   Acc: tensor(0.9227) EO: tensor(0.0143)
Task 11   Acc: tensor(0.7164) EO: tensor(0.0022)
Task 12   Acc: tensor(0.7235) EO: tensor(0.0306)
Task 13   Acc: tensor(0.6856) EO: tensor(0.0066)
Task 14   Acc: tensor(0.8047) EO: tensor(0.0269)
Task 15   Acc: tensor(0.7813) EO: tensor(0.0147)
Task 16   Acc: tensor(0.7880) EO: tensor(0.0186)
Task 0   Acc: tensor(0.7065) EO: tensor(0.0168)
Task 1   Acc: tensor(0.7932) EO: tensor(0.0162)
Task 2   Acc: tensor(0.8763) EO: tensor(0.0126)
Task 3   Acc: tensor(0.8488) EO: 